In [4]:
import pandas as pd
df = pd.read_csv("asu_feedback1.csv", sep = ",", encoding ="UTF-8")
df.head()

,id,comment_text,year,stars,target,neutral,negative
0,1,Ежиный шабаш,2024,4.0,2.0,NaN,NaN
1,2,отучился 4года работаю курьером самоката,2023,1.0,3.0,NaN,NaN
2,3,учусь на протяжении 3 лет мне нравиться узнал ...,2023,4.0,1.0,NaN,NaN
3,4,приятный университет вежливые сотрудники и охр...,2023,5.0,1.0,NaN,NaN
4,5,в гардеробе работают ужасные сотрудники Хамско...,2023,1.0,3.0,NaN,NaN


In [5]:
df['target'].value_counts()

target
1.0    164
3.0     76
2.0     44
Name: count, dtype: int64

In [6]:
df[df['target']==5]

,id,comment_text,year,stars,target,neutral,negative


In [7]:
del df['neutral'], df["negative"]
df = df.dropna()

In [8]:
df

,id,comment_text,year,stars,target
0,1,Ежиный шабаш,2024,4.0,2.0
1,2,отучился 4года работаю курьером самоката,2023,1.0,3.0
2,3,учусь на протяжении 3 лет мне нравиться узнал ...,2023,4.0,1.0
3,4,приятный университет вежливые сотрудники и охр...,2023,5.0,1.0
4,5,в гардеробе работают ужасные сотрудники Хамско...,2023,1.0,3.0
...,...,...,...,...,...
281,282,Хороший ВУЗ,2020,5.0,1.0
282,283,Хороший университет,2019,5.0,1.0
283,284,Низкие цены,2020,5.0,1.0
284,285,В хорошем месте,2019,4.0,1.0


In [9]:
X = df[["comment_text", "year", 'stars']]
Y = df['target']

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

C:\Users\User\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
X_test.shape, y_test.shape

((94, 3), (94,))

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from snowballstemmer import stemmer

# Загрузка русского набора стоп-слов
nltk.download('stopwords')
nltk.download('punkt')
russian_stopwords = stopwords.words('russian')

# Функция для препроцессинга текста
def preprocess_text(text):
    # Токенизация текста
    tokens = word_tokenize(text, language="russian")
    # Приведение к нижнему регистру и удаление стоп-слов
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in russian_stopwords]
    # Стемминг (приводим слова к их основе)
    stemmer_ru = stemmer('russian')
    tokens = [stemmer_ru.stemWord(token) for token in tokens]
    return " ".join(tokens)

# Применим функцию к набору данных
X_pre_train = X_train['comment_text'].apply(preprocess_text)
X_pre_test = X_test['comment_text'].apply(preprocess_text)

print(X_pre_train)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


283                                             низк цен
149                втор альм матер опорн вуз алтайск кра
199                                 хорош региональн вуз
148    законч магистратур знан получ отличн лет пригожда
31           опорн университет город барнаул алтайск кра
                             ...                        
190                                   готов достойн кадр
72                                   многопрофессиональн
108    кабинет договорн отдел так отношен люд видел н...
272    очен развит внеурочн деятельн уч ом курс пок в...
104    алтайск государствен университет говор говор г...
Name: comment_text, Length: 190, dtype: object


In [13]:
print(X_pre_test.shape)

(94,)


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=5000)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['comment_text'])
X_test_tfidf = tfidf_vectorizer.transform(X_test['comment_text'])

print(X_train_tfidf.shape)
print(X_test_tfidf.shape)

(190, 2369)
(94, 2369)


In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Создание и обучение модели логистической регрессии
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Прогнозирование на тестовых данных
predictions = model.predict(X_test_tfidf)

# Оценка модели
report = classification_report(y_test, predictions)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

         1.0       0.55      1.00      0.71        52
         2.0       0.00      0.00      0.00        11
         3.0       0.00      0.00      0.00        31

    accuracy                           0.55        94
   macro avg       0.18      0.33      0.24        94
weighted avg       0.31      0.55      0.39        94



C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Создание и обучение модели логистической регрессии
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Прогнозирование на тестовых данных
predictions = model.predict(X_test_tfidf)

# Оценка модели
report = classification_report(y_test, predictions)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

         1.0       0.62      1.00      0.76        52
         2.0       0.00      0.00      0.00        11
         3.0       0.90      0.29      0.44        31

    accuracy                           0.65        94
   macro avg       0.51      0.43      0.40        94
weighted avg       0.64      0.65      0.57        94



C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Создание и обучение модели логистической регрессии
model = SVC()
model.fit(X_train_tfidf, y_train)

# Прогнозирование на тестовых данных
predictions = model.predict(X_test_tfidf)

# Оценка модели
report = classification_report(y_test, predictions)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

         1.0       0.58      1.00      0.74        52
         2.0       0.00      0.00      0.00        11
         3.0       1.00      0.16      0.28        31

    accuracy                           0.61        94
   macro avg       0.53      0.39      0.34        94
weighted avg       0.65      0.61      0.50        94



C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
